In [24]:
import pandas as pd

final_analysis = pd.DataFrame()
endSemMarks = pd.DataFrame()
iaMarks = pd.DataFrame()
totalMarks = pd.DataFrame()

totalCols = []
endsSemCols = []
iaCols = []

subjectcodes = {}

In [25]:
def exportData(outputpath, **listOfDataFrames):
    for name, df in listOfDataFrames.items():
        df.to_excel(outputpath + '/' + name + '.xlsx')

def checkpass(x,val):
    if x == 'Ab':
        return 0
    return int(int(x)>val)


def checkrange(x, p,q):
    if x == 'Ab':
        return 0
    return int(p < int(str(x).strip()) and int(str(x).strip())< q)

In [26]:
inputpath = 'data/Result-Format1.xlsx'

# READ FILE + DROP NULL COLUMNS
original_file = pd.read_excel(inputpath)

# remove rows wiht all na
original_file.dropna(how = "all", inplace = True)

# remove redundant column
original_file.drop(['Unnamed: 1'], axis = 1, inplace = True)
original_file = original_file.iloc[:, :-1]

# scrap part of original dataframe
original_file1 = original_file.iloc[3:, :]
original_file1 = original_file1.dropna(axis = 'columns', how = 'all')

# regenerating old file for selected columns only
original_file = original_file.loc[:, original_file1.columns]

In [27]:
# Working with Header for further use
curr_header = [str(i) for i in original_file.iloc[2, :].values]
add_details = [str(i) for i in original_file.iloc[3, :].values]

header = ['SEAT', 'TOTAL', 'SGPI', 'RESULT']

for i in range(1, len(curr_header) - 4):
    j = i
    while curr_header[j] == 'nan':
        j = j - 1
    header.insert(-3, (curr_header[j].strip() + '-' + add_details[i].strip()))

header_later = header.copy()
header_later.insert(1, 'NAME')

In [28]:
# Creating a list of subject names and codes and mapping them
subjectlist = [i for i in curr_header if i != 'nan']
subjectcode = [i for i in original_file.iloc[1, :].values[1:-4] if str(i) != 'nan']
subjectcodes = dict(zip(subjectcode, subjectlist))

In [29]:
# Creating a dataframe to work on
df = original_file1.iloc[:, :]
df = df.loc[df['Unnamed: 2'].isna() == False]

# removing CGPA column
df = df.iloc[:, :-1]

df.columns = header

df = df.iloc[3:, :]

# dataframe to store marks
dfmarks = df.iloc[::2]

# dataframe that stores names
dfnames = df.iloc[1::2]

# adding names to the marks dataframe
dfmarks = dfmarks.copy()
dfmarks['NAME'] = dfnames.loc[:, 'SEAT'].values
dfmarks = dfmarks.loc[:, header_later]

dfmarks.loc[:, 'NAME'] = dfmarks.loc[:, 'NAME'].apply(lambda x: x.strip('/ '))

dfmarks.index = pd.RangeIndex(1, len(dfmarks.index) + 1)

In [30]:
# Removing Old Students
def getOld(x):
    if '+' in str(x):
        return x
oldStudents = []
for i in dfmarks.columns[2:-3]:
    tmp = [i for i in dfmarks[i].apply(getOld).values if i]
    try:
        oldStudents.append(min([dfmarks[i].values.tolist().index(tmp[j]) for j in range(len(tmp))]))
    except:
        pass
if len(oldStudents) > 0:
    dfmarks = dfmarks.iloc[:min(oldStudents),]

labstr = subjectcode[-1][:3]
totalsubs = []
for k, v in subjectcodes.items():
    if labstr not in k:
        totalsubs.append(v)
if len(totalsubs) == 0:
    totalsubs = subjectlist.copy()
df_header = list(dfmarks.columns)

# Restructuring The Header
totalCols = df_header[:2] + [str(i.strip() + '-TOT') for i in totalsubs] + df_header[-3:]
endsSemCols = df_header[:2] + [str(i.strip() + '-ESE') for i in totalsubs] + df_header[-3:]
iaCols = df_header[:2] + [str(i.strip() + '-IA') for i in totalsubs] + df_header[-3:]

totalMarks = dfmarks[totalCols]
endSemMarks = dfmarks[endsSemCols]
iaMarks = dfmarks[iaCols]

In [31]:
def formatMarks(x):
    x = str(x)
    if x == 'Ab':
        return int(0)

    elif '*' in x:
        x = int(x[:x.index('\r')])
        if abs(int(x) - 32) < 5:
            return 32
        elif abs(int(x) - 8) < 5:
            return 8
    elif '!' in x:
        return 8
    elif '@' in x:
        return int(x[:x.index('\r')]) + int(x[-1])
    elif x[-1] == 'E' or x[-1] == 'F':
        return int("".join(x[:-1]).strip())

    return int(x)

for i in endSemMarks.columns[2:-3]:
    endSemMarks.loc[:,i] = endSemMarks.loc[:,i].apply(formatMarks)

for i in iaMarks.columns[2:-3]:
    iaMarks.loc[:,i] = iaMarks.loc[:,i].apply(formatMarks)

endSemMarks.loc[:, 'TOTAL'] = endSemMarks.loc[:, 'TOTAL'].apply(formatMarks)
iaMarks.loc[:, 'TOTAL'] = iaMarks.loc[:, 'TOTAL'].apply(formatMarks)
totalMarks.loc[:, 'TOTAL'] = totalMarks.loc[:, 'TOTAL'].apply(formatMarks)

endSemMarks = endSemMarks.loc[:, ~endSemMarks.columns.duplicated()]
iaMarks = iaMarks.loc[:, ~iaMarks.columns.duplicated()]
totalMarks = totalMarks.loc[:, ~totalMarks.columns.duplicated()]

for i in range(2, len(totalMarks.columns)-3):
    totalMarks.iloc[:,i] = endSemMarks.iloc[:,i] + iaMarks.iloc[:,i]

/Users/aayushsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [32]:
endSemMarks

,SEAT,NAME,APPLIED MATHEMATICS III-ESE,THERMODYNAMICS-ESE,STRENGTH OF MATERIALS-ESE,PRODUCTION PROCESS I-ESE,MATERIAL TECHNOLOGY-ESE,TOTAL,SGPI,RESULT
1,13001,ABHIJITH SHAH NITA,34,15,26,41,41,344,2.96,F
2,13002,AGRAWAL AKHILESH SANJAY ANURADHA,39,21,41,48,41,366,3.69,F
3,13003,AGRAWAL RONAK ANIL VARSHA,47,39,64,53,49,471,7.23,P
4,13004,ALDAR ROHAN PANDHARINATH SANGITA,35,21,38,34,37,333,3.12,F
5,13005,AWATE TEJAS PANDURANG RANJANA,0,10,22,45,33,323,3.65,F
6,13006,BHALERAO ASHUTOSH VIJAY AARTI,39,39,54,66,57,517,7.92,P
7,13007,BHARAMBE DAMINI NITIN JYOTI,66,67,65,55,61,584,9.35,P
8,13008,BHOIR CHIRAG PRADEEP PRACHI,69,45,64,45,54,521,8.08,P
9,13009,BHURKE SOHAM NITIN NEHA,51,32,68,34,46,477,7.46,P
10,13010,BOKKA HARISH TATARAO KUSUMA,62,44,46,46,48,492,7.69,P


In [33]:
iaMarks

,SEAT,NAME,APPLIED MATHEMATICS III-IA,THERMODYNAMICS-IA,STRENGTH OF MATERIALS-IA,PRODUCTION PROCESS I-IA,MATERIAL TECHNOLOGY-IA,TOTAL,SGPI,RESULT
1,13001,ABHIJITH SHAH NITA,2,9,4,16,6,344,2.96,F
2,13002,AGRAWAL AKHILESH SANJAY ANURADHA,5,9,9,9,10,366,3.69,F
3,13003,AGRAWAL RONAK ANIL VARSHA,14,14,12,15,15,471,7.23,P
4,13004,ALDAR ROHAN PANDHARINATH SANGITA,5,9,8,9,5,333,3.12,F
5,13005,AWATE TEJAS PANDURANG RANJANA,5,8,8,13,11,323,3.65,F
6,13006,BHALERAO ASHUTOSH VIJAY AARTI,9,15,14,17,14,517,7.92,P
7,13007,BHARAMBE DAMINI NITIN JYOTI,18,17,13,17,17,584,9.35,P
8,13008,BHOIR CHIRAG PRADEEP PRACHI,17,13,12,11,12,521,8.08,P
9,13009,BHURKE SOHAM NITIN NEHA,16,13,14,12,13,477,7.46,P
10,13010,BOKKA HARISH TATARAO KUSUMA,18,14,15,13,14,492,7.69,P


In [34]:
totalMarks

,SEAT,NAME,APPLIED MATHEMATICS III-TOT,THERMODYNAMICS-TOT,STRENGTH OF MATERIALS-TOT,PRODUCTION PROCESS I-TOT,MATERIAL TECHNOLOGY-TOT,TOTAL,SGPI,RESULT
1,13001,ABHIJITH SHAH NITA,36,24,30,57,47,344,2.96,F
2,13002,AGRAWAL AKHILESH SANJAY ANURADHA,44,30,50,57,51,366,3.69,F
3,13003,AGRAWAL RONAK ANIL VARSHA,61,53,76,68,64,471,7.23,P
4,13004,ALDAR ROHAN PANDHARINATH SANGITA,40,30,46,43,42,333,3.12,F
5,13005,AWATE TEJAS PANDURANG RANJANA,5,18,30,58,44,323,3.65,F
6,13006,BHALERAO ASHUTOSH VIJAY AARTI,48,54,68,83,71,517,7.92,P
7,13007,BHARAMBE DAMINI NITIN JYOTI,84,84,78,72,78,584,9.35,P
8,13008,BHOIR CHIRAG PRADEEP PRACHI,86,58,76,56,66,521,8.08,P
9,13009,BHURKE SOHAM NITIN NEHA,67,45,82,46,59,477,7.46,P
10,13010,BOKKA HARISH TATARAO KUSUMA,80,58,61,59,62,492,7.69,P
